In [1]:
import pandas as pd
import os
from tqdm import *
import numpy as np

In [4]:
# variables 
bus_stops = [43901.0]
encoding = 'latin1'

p = r"C:\Data\Aktuell"
n_p = os.path.join(p, 'interim')
n_a = os.path.join(p, 'all')

if not os.path.exists(n_p):
    os.makedirs(n_p)
if not os.path.exists(n_a):
    os.makedirs(n_a)    

files = os.listdir(p)
files = [f for f in files if '.csv' in f]

n_p_files = os.listdir(n_p)
for f in n_p_files:
    os.remove(os.path.join(n_p,f))

In [5]:
df = pd.read_csv(os.path.join(p, files[0]), sep=';', index_col=False)
cols = df.columns
cols_dict = {}
for i, col in enumerate(cols):
    cols_dict[col] = i

usecols_names = ['Wagen', 'Datum', 'Zeit', 'HstCode', 'HstName', 'Ein', 'Aus', 'X', 'Y']
usecols_ids = []
for n in usecols_names:
    usecols_ids.append(cols_dict[n])

In [6]:
df = pd.read_csv(os.path.join(p, files[0]), sep=';', index_col=False, usecols=usecols_ids, encoding=encoding)
df.dropna(inplace=True)
df = df[df['HstCode'].isin(bus_stops)]
total = len(files)

f_n = 0

for i, f in tqdm(enumerate(files[1:]), total = total-1):
    df_new = pd.read_csv(
            os.path.join(p, f), sep=';', index_col=False, usecols=usecols_ids, encoding=encoding)
    df_new.dropna(inplace=True)

    # Select only Sophienstrasse (43901.0)
    df_new = df_new[df_new['HstCode'].isin(bus_stops)]
    df = df.append(df_new)
    if i % 30 == 0:
        df.to_csv(os.path.join(n_p, 'all' + str(f_n) + '.csv'), sep= ';', encoding=encoding)
        df = df[df.index == None]
        f_n += 1

100%|██████████████████████████████████████████████████████████████████████████████| 5866/5866 [04:19<00:00, 22.65it/s]


In [7]:
df_name = 'all'
for stop in bus_stops:
    df_name = df_name + '_' + str(stop)[:-2]
df_name += '.csv'

files = os.listdir(n_p)
df = pd.read_csv(os.path.join(n_p, files[0]), sep=';', index_col=0, encoding=encoding)
for f in tqdm(files[1:]):
    df = df.append(pd.read_csv(os.path.join(n_p, f), sep=';', encoding=encoding))
    
df.to_csv(os.path.join(n_a, df_name), index=False)

100%|███████████████████████████████████████████████████████████████████████████████| 195/195 [00:00<00:00, 221.84it/s]


UnicodeEncodeError: 'ascii' codec can't encode character u'\xdf' in position 11: ordinal not in range(128)

In [ ]:
df.sort_values('Zeit', ascending=False)